In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from operator import index
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from fastai.tabular.all import df_shrink
from google.colab import data_table
from google.colab import files
from google.colab import drive
from sklearn.ensemble import IsolationForest
from sklearn.metrics import accuracy_score



# For LOF
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

testing = '/content/drive/MyDrive/Testing/KDDTest+.csv'
training = '/content/drive/MyDrive/Testing/KDDTrain+.csv'

train = pd.read_csv(training, encoding='utf-8')
test = pd.read_csv(testing, encoding='utf-8')

train = df_shrink(train, skip=[], obj2cat=True, int2uint=True)
test = df_shrink(test, skip=[], obj2cat=True, int2uint=True)

feature_type_map = {f: (train.dtypes.loc[f], test.dtypes.loc[f]) for f in train.columns}

len(train['class'].cat.categories), len(test['class'].cat.categories)

len(train.service.cat.categories), len(test.service.cat.categories)

def train_test_category_union(feature):
  tcl = len(train[feature].cat.categories)
  tcll = len(test[feature].cat.categories)
  feature_cat_union = train[feature].cat.categories.union(test[feature].cat.categories)
  train[feature] = train[feature].cat.set_categories(feature_cat_union)
  test[feature] = test[feature].cat.set_categories(feature_cat_union)

cat = list(train.select_dtypes(include='category').columns.values)

for f in cat:
  train_test_category_union(f)

class_data_train = train['class']
#one hot encoding on all nominal categories except class
test = pd.get_dummies(test, columns=['protocol_type', 'service', 'flag', 'land', 'logged_in', 'is_host_login', 'is_guest_login'])
train = pd.get_dummies(train, columns=['protocol_type', 'service', 'flag', 'land', 'logged_in', 'is_host_login', 'is_guest_login'])

#label encoding for class column since it is the target variable
le = LabelEncoder()
train['classification'] = le.fit_transform(train['class'])
test['classification'] = le.fit_transform(test['class'])
train.drop('class', axis=1, inplace=True)
test.drop('class', axis=1, inplace=True)

#robust scaling for normalization step
robust_scaler1 = RobustScaler()
robust_scaler2 = RobustScaler()
#data1 = test.values[:, :-1]
data1 = test.values[:, :]
#data2 = train.values[:, :-1]
data2 = train.values[:, :]
data1 = robust_scaler1.fit_transform(data1)
data2 = robust_scaler2.fit_transform(data2)
#converting array back to dataframe
test = pd.DataFrame(data1)
train = pd.DataFrame(data2)

# X_train = train.drop('classification', axis=1)
# y_train = train['classification']
# X_test = test.drop('classification', axis=1)
# y_test = test['classification']


# # Standard Scaling for normalization step
# standard_scaler1 = StandardScaler()
# standard_scaler2 = StandardScaler()

# # use all columns (including classification, but we fix that below)
# data1 = test.values[:, :]
# data2 = train.values[:, :]

# # fit on train, transform both
# data1 = standard_scaler1.fit_transform(data1)
# data2 = standard_scaler2.fit_transform(data2)


# # convert back to DataFrame
# # test = pd.DataFrame(data1, columns=test.columns)
# test = pd.DataFrame(data1)
# # train = pd.DataFrame(data2, columns=train.columns)
# train = pd.DataFrame(data2)

# Preprocessing ends here

# print(test[126]).to_string(index=True)

In [ ]:
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import classification_report, accuracy_score

# Separates features and labels
X_train = train.drop(train.columns[-1], axis=1)  # all but last col
y_train = train[train.columns[-1]]               # last col is classification
X_test = test.drop(test.columns[-1], axis=1)
y_test = test[test.columns[-1]]

# Initialize the LOF
lof = LocalOutlierFactor(n_neighbors=3, novelty=True, contamination=0.00522) # 3 / 0.00522 : 0.7688520227111426

# Fit on training data
lof.fit(X_train)

# Predict on test data
y_pred = lof.predict(X_test)

# LOF outputs: -1 = anomaly, 1 = normal
# We need: 0 = anomaly, 1 = normal
# Convert predictions
y_pred = np.where(y_pred == -1, 0, 1)

# Evaluate
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=["Anomaly (0)", "Normal (1)"]))

Accuracy: 0.7688076650106459

Classification Report:
               precision    recall  f1-score   support

 Anomaly (0)       0.88      0.69      0.77     12833
  Normal (1)       0.68      0.87      0.76      9711

    accuracy                           0.77     22544
   macro avg       0.78      0.78      0.77     22544
weighted avg       0.79      0.77      0.77     22544



In [ ]:
#outliers_train = len(train[train[126]==-1])/float(len(train[train[126]==0]))
#outliers_test = len(test[test[126]==0])/float(len(test))
#print(outliers_test)
#detection_model_test = IsolationForest(n_estimators=356, contamination=0.000001, random_state=60)
# estimate = 251
# contam = .41450284628489975
# samples =4020
# state = 94
# for i in range(200):
#   detection_model = IsolationForest(n_estimators=estimate, contamination=contam, random_state=state, max_samples=samples)
#   detection_model.fit(train)
#   y_test_pred = detection_model.predict(test)
#   y_test_pred[y_test_pred == 1] = 1
#   y_test_pred[y_test_pred == -1] = 0
#   print(accuracy_score(target_test, y_test_pred))
#   #state = state + 1
#   #estimate = estimate - 1
#   contam = contam - .0000000005000
#   #samples = samples + 10
# #detection_model.fit(test)

In [ ]:
# #plt.figure(figsize=(50,50))
# plt.figure()
# #plt.ylim(0, 50)
# #plt.xlim(0,1000)
# plt.scatter(test.iloc[y_test_pred==0,0].values, test.iloc[y_test_pred==0,1].values, c='blue', label='Normal')
# plt.scatter(test.iloc[y_test_pred==1,0].values, test.iloc[y_test_pred==1,1].values, c='red', label='Anomaly')
# plt.legend()
# plt.show()